---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    ut=pd.read_csv('university_towns.txt',header=None,names=['data'])
    s=0
    State=[]
    RegionName=[]
    for i in range(0,len(ut)):
        if 'edit' in ut['data'][i]:
            s=ut['data'][i][:ut['data'][i].find('[')]
            ut['data'][i]=ut['data'][i][:ut['data'][i].find('(')]
            ut['data'][i]=ut['data'][i][:ut['data'][i].find('[')+1]
        State.append(s)
        RegionName.append(ut['data'][i][:ut['data'][i].find('(')].strip())
    ss=pd.DataFrame(RegionName,State).reset_index()
    ss.columns=['State','RegionName']
    df1=ss[ss['State']!=ss['RegionName']]
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df2=df1.reset_index().drop('index',axis=1)
       
    return df2

In [13]:
def get_recession_start():
    url2='https://www.bea.gov/system/files/2018-05/gdplev_0.xlsx'
    df2=pd.read_excel(url2, skiprows=[1,2,3,4,5,6],header=1, index=0, names=['year','GDP1','GDPch1','blank','yearqtr','GDP2','GDPch2','na'])
    yr=df2[['year','GDP1','GDPch1']].loc[0:89]
    yrqtr=df2[df2['yearqtr']>='2000q1'][['yearqtr','GDP2','GDPch2']].reset_index().drop('index',axis=1)
    yrqtr['yearqtr']=[x.lower() for x in yrqtr['yearqtr']]
    yrqtr['type']=''
    for i in range(0,len(yrqtr)-1):
        if yrqtr['GDPch2'].loc[i+1]>yrqtr['GDPch2'].loc[i]:
            yrqtr.loc[i+1,'type']='growth'
        else:
            yrqtr.loc[i+1,'type']='decline'
    
    res_start=[]
    for i in range(0,len(yrqtr)-1):
        if yrqtr['type'].iloc[i]=='decline' and yrqtr['type'].iloc[i+1]=='decline' and yrqtr['type'].iloc[i+2]=='growth' and yrqtr['type'].iloc[i+3]=='growth':
            j=i
            while yrqtr['type'].loc[j]=='decline':
                j-=1
                
            res_start.append(yrqtr['yearqtr'].iloc[j+1])

    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return res_start[0]

In [19]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    url2='https://www.bea.gov/system/files/2018-05/gdplev_0.xlsx'
    df2=pd.read_excel(url2, skiprows=[1,2,3,4,5,6],header=1, index=0, names=['year','GDP1','GDPch1','blank','yearqtr','GDP2','GDPch2','na'])
    yr=df2[['year','GDP1','GDPch1']].loc[0:89]
    yrqtr=df2[df2['yearqtr']>='2000q1'][['yearqtr','GDP2','GDPch2']].reset_index().drop('index',axis=1)
    yrqtr['yearqtr']=[x.lower() for x in yrqtr['yearqtr']]    
    yrqtr['type']=''
    for i in range(0,len(yrqtr)-1):
        if yrqtr['GDPch2'].loc[i+1]>yrqtr['GDPch2'].loc[i]:
            yrqtr.loc[i+1,'type']='growth'
        else:
            yrqtr.loc[i+1,'type']='decline'
    res_start=[]
    for i in range(0,len(yrqtr)-1):
        if yrqtr['type'].loc[i]=='decline' and yrqtr['type'].loc[i+1]=='decline' and yrqtr['type'].loc[i+2]=='growth' and yrqtr['type'].loc[i+3]=='growth':
            res_start.append(yrqtr['yearqtr'].loc[i+3])
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return res_start[0]

In [21]:
def get_recession_bottom():
    url2='https://www.bea.gov/system/files/2018-05/gdplev_0.xlsx'
    df2=pd.read_excel(url2, skiprows=[1,2,3,4,5,6],header=1, index=0, names=['year','GDP1','GDPch1','blank','yearqtr','GDP2','GDPch2','na'])
    yr=df2[['year','GDP1','GDPch1']].loc[0:89]
    yrqtr=df2[df2['yearqtr']>='2000q1'][['yearqtr','GDP2','GDPch2']].reset_index().drop('index',axis=1)
    yrqtr['yearqtr']=[x.lower() for x in yrqtr['yearqtr']]    
    yrqtr['type']=''
    for i in range(0,len(yrqtr)-1):
        if yrqtr['GDPch2'].loc[i+1]>yrqtr['GDPch2'].loc[i]:
            yrqtr.loc[i+1,'type']='growth'
        else:
            yrqtr.loc[i+1,'type']='decline'
    res_start=[]
    for i in range(0,len(yrqtr)-1):
        if yrqtr['type'].loc[i]=='decline' and yrqtr['type'].loc[i+1]=='decline' and yrqtr['type'].loc[i+2]=='growth' and yrqtr['type'].loc[i+3]=='growth':
            res_start.append(yrqtr['yearqtr'].loc[i+1])
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return res_start[0]
    
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''

In [23]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    h=pd.read_csv('City_Zhvi_AllHomes.csv')
    h=h.drop(['RegionID','Metro','CountyName','SizeRank'],axis=1).set_index(['RegionName','State'])
    for i in h.columns:
        if int(i[:4])<2000:
            h=h.drop(i, axis=1)
    def yrqtr(x):
        if x[-2:] in ['01','02','03']:
            return x[:4]+'q1'
        elif x[-2:] in ['04','05','06']:
            return x[:4]+'q2'
        elif x[-2:] in ['07','08','09']:
            return x[:4]+'q3'
        elif x[-2:] in ['10','11','12']:
            return x[:4]+'q4'
        else:
            return x

    h.columns=h.columns.map(yrqtr)
    t=h.transpose()
    v=t.groupby(level=0).mean().transpose()
    v=v.reset_index()
    v['State']=[states[x] for x in v['State']]
    v=v.set_index(['State','RegionName'])
    return v

In [26]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    ut=get_list_of_university_towns()
    hd=convert_housing_data_to_quarters()
    st=get_recession_start()
    bm=get_recession_bottom()
    ed=get_recession_end()
    s=[]
    for key, value in states.items():
        s.append([value,key])
    rstate = {v:k for v,k in s}
    ut=ut.reset_index()
    hd=hd.reset_index()
    ut['Stateind']=[rstate[x] for x in ut['State']]
    hd['Stateind']=[rstate[y] for y in hd['State']]

    ut['cat']=ut['RegionName']+ut['Stateind']
    hd['cat']=hd['RegionName']+hd['Stateind']
    hd['diff']=hd[bm]-hd[st]
    hdr=hd[['cat','diff']]
    mydata=hdr.merge(ut,how='outer',left_on=['cat'], right_on=['cat'])
    mydata=mydata[pd.isnull(mydata['diff'])==False]
    univ=mydata[pd.isnull(mydata['index'])==False]
    nonuniv=mydata[pd.isnull(mydata['index'])==True]
    from scipy import stats
    t=stats.ttest_ind(univ['diff'],nonuniv['diff'])
    if t[1]<=0.01:
        different=True
        better='university town'
    else:
        different=False
        better='non-university town'

    return (different, t[1],better )